### **02_model_training.ipynb**
### **Model Training Pipeline**

* ##### 01 - Install packages
* ##### 02 - Import packages
* ##### 03 - Create tasks
* ##### 04 - Create pipeline
* ##### 05 - Create pipeline yaml
* ##### 06 - Create pipeline run
* ##### 07 - Run inference

### 01 - Install packages

In [1]:
!pip install kfp requests tensorflow==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 202.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 221.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 190.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 232.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 172.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 203.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 197.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 134.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 181.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 199.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 154.2 MB/s eta 0:00:00
     ━━━━━━━━━

### 02 - Import packages

In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import json
import kfp
import kfp.kubernetes as kubernetes
import requests
import tensorflow as tf

from PIL import Image

from components.create_model     import create_model
from components.delete_artifacts import delete_artifacts
from components.download_dataset import download_dataset
from components.evaluate_model   import evaluate_model
from components.prepare_dataset  import prepare_dataset
from components.train_model      import train_model
from components.upload_artifacts import upload_artifacts
from components.upload_model     import upload_model

2024-06-05 20:44:37.568530: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 20:44:37.609010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 20:44:37.609084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 20:44:37.610484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 20:44:37.617929: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 20:44:37.618781: I tensorflow/core/platform/cpu_feature_guard.cc:1

### 03 - Create tasks

In [3]:
task_base_image = 'registry.access.redhat.com/ubi9/python-311'

In [4]:
download_dataset_op = kfp.dsl.component(
    func       = download_dataset,
    base_image = task_base_image
)

In [5]:
prepare_dataset_op = kfp.dsl.component(
    func       = prepare_dataset,
    base_image = task_base_image
)

In [7]:
create_model_op = kfp.dsl.component(
    func                = create_model,
    base_image          = task_base_image,
    packages_to_install = ['tensorflow==2.15.0']
)

In [8]:
train_model_op = kfp.dsl.component(
    func                = train_model,
    base_image          = task_base_image,
    packages_to_install = ['tensorflow==2.15.0']
)

In [9]:
evaluate_model_op = kfp.dsl.component(
    func                = evaluate_model,
    base_image          = task_base_image,
    packages_to_install = ['tensorflow==2.15.0']
)

In [10]:
upload_artifacts_op = kfp.dsl.component(
    func                = upload_artifacts,
    base_image          = task_base_image,
    packages_to_install = ['boto3']
)

In [11]:
upload_model_op = kfp.dsl.component(
    func                = upload_model,
    base_image          = task_base_image,
    packages_to_install = ['boto3', 'openvino']
)

In [12]:
delete_artifacts_op = kfp.dsl.component(
    func       = delete_artifacts,
    base_image = task_base_image
)

### 04 - Create pipeline

In [13]:
pipeline_name        = '02_model_training'
pipeline_description = 'Model Training Pipeline'

In [31]:
@kfp.dsl.pipeline(
    name        = pipeline_name,
    description = pipeline_description
)
def pipeline(
    s3_service_name      : str,
    s3_endpoint_url      : str,
    s3_access_key_id     : str,
    s3_secret_access_key : str,
    s3_region            : str,
    s3_bucket            : str
):

    import os

    create_pvc_task = kubernetes.CreatePVC(
        pvc_name_suffix    = '-pipeline-pvc',
        size               = '1Gi',
        access_modes       = ['ReadWriteOnce'],
        storage_class_name = 'mystorageclassname'
    )

    pvc_directory = os.path.join('/', 'pipeline')
    pvc_name      = create_pvc_task.outputs['name']

    download_dataset_task = download_dataset_op()
    kubernetes.mount_pvc(
        task       = download_dataset_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    download_dataset_task.after(create_pvc_task)

    prepare_dataset_task = prepare_dataset_op()
    kubernetes.mount_pvc(
        task       = prepare_dataset_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    prepare_dataset_task.after(download_dataset_task)

    create_model_task = create_model_op()
    kubernetes.mount_pvc(
        task       = create_model_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    create_model_task.after(prepare_dataset_task)

    train_model_task = train_model_op()
    kubernetes.mount_pvc(
        task       = train_model_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    train_model_task.after(create_model_task)

    evaluate_model_task = evaluate_model_op()
    kubernetes.mount_pvc(
        task       = evaluate_model_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    evaluate_model_task.after(train_model_task)

    upload_artifacts_task = upload_artifacts_op(
        s3_service_name      = s3_service_name,
        s3_endpoint_url      = s3_endpoint_url,
        s3_access_key_id     = s3_access_key_id,
        s3_secret_access_key = s3_secret_access_key,
        s3_region            = s3_region,
        s3_bucket            = s3_bucket,
        pipeline_name        = pipeline_name
    )
    kubernetes.mount_pvc(
        task       = upload_artifacts_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    upload_artifacts_task.after(evaluate_model_task)

    upload_model_task = upload_model_op(
        s3_service_name      = s3_service_name,
        s3_endpoint_url      = s3_endpoint_url,
        s3_access_key_id     = s3_access_key_id,
        s3_secret_access_key = s3_secret_access_key,
        s3_region            = s3_region,
        s3_bucket            = s3_bucket,
        pipeline_name        = pipeline_name
    )
    kubernetes.mount_pvc(
        task       = upload_model_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    upload_model_task.after(evaluate_model_task)

    delete_artifacts_task = delete_artifacts_op()
    kubernetes.mount_pvc(
        task       = delete_artifacts_task,
        pvc_name   = pvc_name,
        mount_path = pvc_directory,
    )
    delete_artifacts_task.after(upload_artifacts_task, upload_model_task)

    kubernetes.DeletePVC(pvc_name = pvc_name).after(delete_artifacts_task)

### 05 - Create pipeline yaml

In [32]:
pipeline_package_path = os.path.join('yaml', f'{ pipeline_name }.yaml')

In [33]:
kfp.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path  = pipeline_package_path
)

### 06 - Create pipeline run

In [34]:
kubeflow_host  = 'https://ds-pipeline-dspa-my-ds-project.apps.cluster-clr5b.dynamic.redhatworkshops.io'
kubeflow_token = 'sha256~gNWgsF3ibmwhmRKHK8aoI4FDs-YGrPwzTnsYoXFCHq0'

In [35]:
pipeline_arguments = {
    's3_service_name'      : 's3',
    's3_endpoint_url'      : 'https://minio-api-minio.apps.cluster-clr5b.dynamic.redhatworkshops.io:',
    's3_access_key_id'     : 'minio',
    's3_secret_access_key' : 'minio123',
    's3_region'            : 'nordeste',
    's3_bucket'            : 'pipelines',
}

In [36]:
kfp.client.Client(host = kubeflow_host, existing_token = kubeflow_token).create_run_from_pipeline_package(
    pipeline_file = pipeline_package_path,
    arguments     = pipeline_arguments
)

RunPipelineResult(run_id=343a456f-9858-4b1a-ba10-195b74b69f9c)

### 07 - Run inference

In [ ]:
inference_endpoint       = '<inference_endpoint>'
inference_endpoint_token = '<inference_endpoint_token>'

In [ ]:
def run_inference(image):

    Image.open(image).show()

    image = tf.keras.utils.load_img(image, target_size = (160, 160))
    image = tf.keras.utils.img_to_array(image)
    image = image.tolist()
    image = [image]

    headers = {
        'Content-Type'  : 'application/json',
        'Authorization' : f'Bearer { inference_endpoint_token }'
    }

    payload = json.dumps({
        'inputs' : [
            {
                'name'     : 'layer_0_input',
                'datatype' : 'FP32',
                'data'     : image,
                'shape'    : [1, 160, 160, 3]
            }
        ]
    })

    response = requests.post(url = inference_endpoint, headers = headers, data = payload)
    response = json.loads(response.text)

    prediction_score = response['outputs'][0]['data'][0]
    prediction       = 'cat' if prediction_score < 0.5 else 'dog'

    print(f'prediction : { prediction }')
    print(f'score      : { prediction_score }')

In [ ]:
run_inference('images/cat.png')

In [ ]:
run_inference('images/dog.png')